In [35]:
import torch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [30]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

In [31]:
import pandas as pd
from datasets import Dataset

# Load your samples from the CSV file
data_path = "./data/raw/samples.csv"
df = pd.read_csv(data_path)

df['input'] = df['dart']
df['target'] = df['javascript']

# prefix every input with 'translate from dart to javascript:'
df['input'] = 'translate from dart to javascript: ' + df['input']

# Create a Dataset object from the DataFrame
dataset = Dataset.from_pandas(df)

# Tokenize the dataset
def preprocess_function(examples):
    inputs = [ex for ex in examples['input']]
    targets = [ex for ex in examples['target']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/212 [00:00<?, ? examples/s]

c:\Users\Ayush\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [32]:
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

c:\Users\Ayush\anaconda3\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/159 [00:00<?, ?it/s]

  0%|          | 0/53 [00:01<?, ?it/s]

{'eval_loss': 0.18374280631542206, 'eval_runtime': 113.4469, 'eval_samples_per_second': 1.869, 'eval_steps_per_second': 0.467, 'epoch': 1.0}


  0%|          | 0/53 [00:00<?, ?it/s]

{'eval_loss': 0.0879971832036972, 'eval_runtime': 159.2284, 'eval_samples_per_second': 1.331, 'eval_steps_per_second': 0.333, 'epoch': 2.0}


  0%|          | 0/53 [00:00<?, ?it/s]

{'eval_loss': 0.07505728304386139, 'eval_runtime': 128.1624, 'eval_samples_per_second': 1.654, 'eval_steps_per_second': 0.414, 'epoch': 3.0}
{'train_runtime': 3055.4236, 'train_samples_per_second': 0.208, 'train_steps_per_second': 0.052, 'train_loss': 0.38001371179736637, 'epoch': 3.0}


TrainOutput(global_step=159, training_loss=0.38001371179736637, metrics={'train_runtime': 3055.4236, 'train_samples_per_second': 0.208, 'train_steps_per_second': 0.052, 'total_flos': 387297218396160.0, 'train_loss': 0.38001371179736637, 'epoch': 3.0})

In [36]:
gpumodel = model.to(device)

In [38]:
print(df['dart'][0])

  List<int> twoSum(List<int> nums, int target) {
    Map<int, int> map = {};
    for (int i = 0; i < nums.length; i++) {
      int complement = target - nums[i];
      if (map.containsKey(complement)) {
        return [map[complement]!, i];
      }
      map[nums[i]] = i;
    }
    throw ArgumentError("No two sum solution");
  }
  


In [43]:
text = '''translate from dart to javascript: int add(int a, int b) {
    return a + b;
}'''

input_ids = tokenizer(text, return_tensors="pt").to(device).input_ids

# simply generate a single sequence
generated_ids = gpumodel.generate(input_ids, max_length=50)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

var add = function(a, b) {
    return a + b;
    };



In [44]:
# Save the model and tokenizer
gpumodel.save_pretrained("./models")
tokenizer.save_pretrained("./models")

('./models\\tokenizer_config.json',
 './models\\special_tokens_map.json',
 './models\\vocab.json',
 './models\\merges.txt',
 './models\\added_tokens.json')